In [1]:
import json
from datetime import date
from typing import Dict, List, Optional

In [2]:
DEFAULT_HABITS = ["Leitura", "Estudar", "Beber água", "Treinar"]
POINTS_PER_HABIT = 10
LEVEL_UP_THRESHOLD = 100

In [ ]:
import json
from datetime import date
from typing import Dict, List, Optional

DEFAULT_HABITS = ["Exercício", "Leitura"]
POINTS_PER_HABIT = 10
LEVEL_UP_THRESHOLD = 100

class HabitTracker:
    def __init__(self, path: str = "habits.json", habits: Optional[List[str]] = None):
        self.path = path
        self.habits = habits or DEFAULT_HABITS
        self.data: Dict[str, object] = {
            "habits": {h: [] for h in self.habits},
            "points": 0,
            "level": 1,
        }
        self._load()

    def _load(self) -> None:
        try:
            with open(self.path, "r", encoding="utf-8") as f:
                raw = json.load(f)
        except FileNotFoundError:
            raw = {}
        raw.setdefault("habits", {})
        for habit in self.habits:
            raw["habits"].setdefault(habit, [])
        raw.setdefault("points", 0)
        raw.setdefault("level", 1)
        self.data = raw
        self._save()

    def _save(self) -> None:
        with open(self.path, "w", encoding="utf-8") as f:
            json.dump(self.data, f, ensure_ascii=False, indent=2)

    def mark_done(self, habit: str, on_date: Optional[str] = None) -> None:
        habits = self.data["habits"]
        if habit not in habits:
            raise ValueError(f"Hábito '{habit}' não encontrado")
        if on_date is None:
            on_date = date.today().isoformat()
        if on_date not in habits[habit]:
            habits[habit].append(on_date)
            self.data["points"] += POINTS_PER_HABIT
            while self.data["points"] >= self.data["level"] * LEVEL_UP_THRESHOLD:
                self.data["level"] += 1
            self._save()

    def get_status(self, on_date: Optional[str] = None) -> Dict[str, bool]:
        if on_date is None:
            on_date = date.today().isoformat()
        habits = self.data["habits"]
        return {habit: on_date in dates for habit, dates in habits.items()}

    def get_stats(self) -> Dict[str, int]:
        return {"points": int(self.data["points"]), "level": int(self.data["level"])}

def main() -> None:
    tracker = HabitTracker()
    while True:
        print("1) Marcar hábito como concluído")
        print("2) Mostrar status de hoje")
        print("3) Mostrar estatísticas")
        print("4) Sair")
        choice = input("Escolha: ").strip()
        if choice == "1":
            for idx, habit in enumerate(tracker.habits, start=1):
                print(f"{idx}) {habit}")
            try:
                idx = int(input("Qual hábito? ")) - 1
                habit = tracker.habits[idx]
            except (ValueError, IndexError):
                print("Opção inválida")
                continue
            tracker.mark_done(habit)
            stats = tracker.get_stats()
            print(f"{habit} marcado para hoje! Pontos: {stats['points']} | Nível: {stats['level']}")
        elif choice == "2":
            status = tracker.get_status()
            for habit, done in status.items():
                print(f"{habit}: {'✔️' if done else '✘'}")
        elif choice == "3":
            stats = tracker.get_stats()
            print(f"Pontos: {stats['points']} | Nível: {stats['level']}")
        elif choice == "4":
            break
        else:
            print("Opção inválida")

if __name__ == "__main__":
    main()


1) Marcar hábito como concluído
2) Mostrar status de hoje
3) Mostrar estatísticas
4) Sair
